<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/0516_HW_Convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **합성곱 신경망**

딥러닝의 역전파 : 순전파 과정에 따라 계산된 오차 정보가 신경망의 모든 노드(출력층 > 은닉층 > 입력층)로 전송됨

- 이런 계산 과정은 복잡하고 많은 자원을 요구 + 계산 시간도 오래 걸림

위와 같은 문제를 해결하고자 하는 것이 바로 **합성곱 신경망!**
- 이미지 전체를 한 번에 계산하는 것이 아니라 이미지의 국소적 부분을 계산함으로써 시간과 자원을 절약하여 이미지의 세밀한 부분까지 분석할 수 있음

## **합성곱층의 필요성**

합성곱 신경망은 이미지나 영상을 처리하는 데 유용

이미지 분석은 이미지를 펼쳐서(flattening) 각 픽셀에 가중치를 곱하여 은닉층으로 전달하게 되는데, 이미지를 펼쳐서 분석하면 데이터의 공간적 구조를 무시하게 됨
- 이를 방지하려고 도입된 것이 합성곱층!

![image analysis](https://thebook.io/img/080263/142.jpg)

## **합성곱 신경망 구조**

![cnn structure](https://thebook.io/img/080263/143.jpg)

1. 입력층
2. 합성곱층
3. 풀링층
4. 완전 연결층
5. 출력층

- 합성곱층 > 풀링층 : 입력 이미지의 주요 특성 벡터(feature vector)를 추출
- 완전 연결층 : 특성 벡터를 1차원 벡터로 변환
- 출력층 : 활성화 함수(softmax)를 사용하여 최종 결과를 출력

### **1. 입력층**

입력 이미지 데이터가 최초로 거치게 되는 계층
- 이미지는 단순 1차원의 데이터가 아닌 높이(height), 너비(width), 채널(channel)의 값을 갖는 3차원 데이터
- 채널 : gray scale인 경우 1, RGB(컬러)인 경우 3 값을 가짐


### **2. 합성곱층**

입력 데이터에서 특성을 추출하는 역할을 수행

[특성 추출 방법]
- 입력 이미지가 들어왔을 때 이미지에 대한 특성을 감지하기 위해 커널(kernel)이나 필터를 사용함
- 커널/필터가 이미지의 모든 영역을 훑으면서 특성을 추출하고, 이때 추출된 결과물이 특성맵(feature map)!
    - 커널은 3x3, 5x5 크기로 적용되는 것이 일반적
    - 스트라이드(stride)라는 지정된 간격에 따라 순차적으로 이동

[특성 추출 과정]
1. 입력 이미지에 3x3 필터 적용
    - 입력 이미지와 필터를 포개어 놓고 대응되는 숫자끼리 곱한 후 모두 더한다

2. 필터가 1만큼 이동 (stride=1)

3. 필터가 1만큼 두 번째 이동 (마지막까지 반복)

4. 새로운 특성 맵 완성

즉, 커널은 스트라이드 간격만큼 순회하면서 모든 입력값과의 합성곱 연산으로 새로운 특성 맵을 만든다!

*(예시) 원본 (6,6,1) -> (4,4,1) 크기의 특성 맵, 3x3 크기의 커널, 스트라이드 1

*마지막 값(채널) = 1 : gray scale을 의미*

![fe1](https://thebook.io/img/080263/145_1.jpg)

![fe2](https://thebook.io/img/080263/145_2.jpg)

![fe3](https://thebook.io/img/080263/145_3.jpg)

![fe4](https://thebook.io/img/080263/146_3.jpg)


**컬러 이미지 합성곱**

- 그레이스케일 이미지와의 차이점 : 필터 채널이 3 + RGB 각각에 서로 다른 가중치로 합성곱을 적용한 후 결과를 더해 줌
    - 필터 채널이 3이라고 해서 필터 개수가 3개인 것이 아님, 1개임

![color image cnn](https://thebook.io/img/080263/147.jpg)

**필터가 두 개 이상인 합성곱**

필터가 2개 이상이면 필터 각각이 특성 추출 결과의 채널이 됨 (계산은 이전과 동일)

![cnn2](https://velog.velcdn.com/images/younguk072023/post/ed9ce155-1a45-4110-96e5-f06316223c1d/image.png)

합성곱층 요약

- **입력 데이터** : $W_1 * H_1 * D_1$ (가로 * 세로 * 채널/깊이)
- **하이퍼파라미터**
    - 필터 개수 : $K$
    - 필터 크기 : $F$
    - 스트라이드 : $S$
    - 패딩 : $P$
- **출력 데이터**
    - $W_2 = (W_1-F+2P)/S+1$
    - $H_2 = (H1-F+2P)/S+1$
    - $D_2 = K$

### **3. 풀링층**

합성곱층과 유사하게 특성 맵의 차원을 다운 샘플링(이미지 축소)하여 연산량을 감소시키고, 주요한 특성 벡터를 추출하여 학습을 효과적으로 할 수 있도록 함

[풀링의 두 가지 연산]

- 최대 풀링(max pooling) : 대상 영역에서 최댓값을 추출
- 평균 풀링(average pooling) : 대상 영역에서 평균을 반환

대부분의 합성곱 신경망에서는 최대 풀링이 사용되는데, 평균 풀링은 각 커널값을 평규노하시켜 중요한 가중치를 갖는 값의 특성이 희미해질 수 있기 때문

**최대/평균 풀링 과정**

- 대상 영역에서 최댓값을 선택 / 평균을 계산
- 스트라이드만큼 이동시켜 그 영역에서 최댓값을 선택 / 평균을 계산

*(예시) 커널 2x2, 스트라이드 2*
![pooling](https://velog.velcdn.com/images/younguk072023/post/d2421615-9b23-42e6-af63-f2baf71ba906/image.png)


풀링 요약 (최대/평균 풀링의 파라미터는 동일)

- **입력 데이터** : $W_1 * H_1 * D_1$ (가로 * 세로 * 채널/깊이)
- **하이퍼파라미터**
    - 필터 크기 : $F$
    - 스트라이드 : $S$

- **출력 데이터**
    - $W_2 = (W_1-F)/S+1$
    - $H_2 = (H1-F)/S+1$
    - $D_2 = D_1$

### **4. 완전연결층**

합성곱층과 풀링층을 거치면서 차원이 축소된 특성 맵은 최종적으로 완전연결층(fully connected layer)으로 전달됨

이 과정에서 이미지는 3차원 벡터에서 1차원 벡터로 펼쳐지게(flatten) 됨

![fully_cnnd_layer](https://velog.velcdn.com/images/younguk072023/post/f35e8ee4-0665-4267-96d5-f84624d7e36e/image.png)

### **5. 출력층**

출력층에서는 소프트맥스 활성화 함수가 사용되므로 입력받은 값을 0~1 사이의 값으로 출력함
- 따라서 마지막 출력층의 소프트맥스 함수를 사용하여 이미지가 각 레이블에 속할 확률 값이 출력되며, 이때 가장 높은 확률 값을 갖는 레이블이 최종 값으로 선정됨

## **1D, 2D, 3D 합성곱**

이동하는 방향의 수와 출력 형태에 따라 1D, 2D, 3D 합성곱으로 분류할 수 있음

### **1D 합성곱**

필터가 시간을 축으로 좌우로만 이동할 수 있는 합성곱
- 입력(W)과 필터(k)에 대한 출력은 W가 됨
- 그래프 곡선을 완화할 때 많이 사용됨

![1d cnn](https://velog.velcdn.com/images/younguk072023/post/d7b85e2e-d038-4377-9016-5a1461dfe787/image.png)

### **2D 합성곱**

필터가 두 방향으로 움직이는 형태
- 입력(W, H)과 필터(k, k)에 대한 출력은 (W, H)가 되며, 출력 형태는 2D 행렬

![2d cnn](https://velog.velcdn.com/images/younguk072023/post/f73ca995-fd89-4dce-a1a8-18800750b4be/image.png)

### **3D 합성곱**

필터가 움직이는 방향이 세 개인 형태
- 입력(W, H, L)에 대해 필터(k, k, d)를 적용하면 출력으로 (W, H, L)을 가짐
- 출력은 3D 형태이고, 이때 **d < L을 유지**하는 것이 중요!

![3d cnn](https://velog.velcdn.com/images/younguk072023/post/74306a90-6782-4c90-bc75-26a3991e80db/image.png)

### **3D 입력을 갖는 2D 합성곱**

입력이 3D 형태임에도 출력 형태가 3D가 아닌 2D 행렬을 취하는 형태
- 필터에 대한 길이(d)가 입력 채널의 길이(L)와 같은 경우
- 입력(W, H, L)에 필터(k, k, L)을 적용하면 출력은 (W, H)가 됨
- 대표적으로 LeNet-5와 VGG가 있음

![3d_2d_cnn](https://velog.velcdn.com/images/younguk072023/post/9cdabca4-6540-4a9a-9c87-952ebbc6a0fe/image.png)

### **1x1 합성곱**

3D 형태로 입력됨
- 입력(W, H, L)에 필터(1, 1, L)를 적용하면 출력은 (W, H)가 됨
- 채널 수를 조정해서 연산량이 감소되는 효과가 있음
- 대표적으로 GoogLeNet

![1x1_cnn](https://velog.velcdn.com/images/younguk072023/post/faf4e3f7-69af-44af-9cb0-b50b7dffcd5f/image.png)